In [ ]:
!pip install PyPDF2
!pip install nltk
!pip install pandas
!pip install sumy
!pip install spacy

import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

import spacy
nlp = spacy.load('en_core_web_sm')

from google.colab import drive
drive.mount('/content/drive')

import os
import PyPDF2
import re
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
import pandas as pd
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def extract_job_title(file_path):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        first_page = reader.pages[0]
        text = first_page.extract_text()

        title_match = re.search(r'^(.*)', text, re.MULTILINE)
        if title_match:
            return title_match.group(1).strip()
        else:
            return None

def extract_job_description(text):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 3)
    return ' '.join(str(sentence) for sentence in summary)

def process_resume(file_path, character_traits):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)

        text = ""
        for page in reader.pages:
            text += page.extract_text()

    lemmatizer = WordNetLemmatizer()

    trait_scores = {}
    for trait, trait_data in character_traits.items():
        keywords = trait_data['keywords']
        weight = trait_data['weight']

        lemmatized_text = " ".join([lemmatizer.lemmatize(word.lower()) for word, tag in pos_tag(text.split()) if tag.startswith('V') or tag.startswith('N')])
        trait_count = sum(1 for keyword in keywords if re.search(r'\b' + re.escape(lemmatizer.lemmatize(keyword)) + r'\b', lemmatized_text, re.IGNORECASE))

        score = min(trait_count * weight, 3)
        trait_scores[trait] = score

    job_description = extract_job_description(text)

    return trait_scores, job_description

def process_directory(directory, character_traits):
    all_trait_scores = defaultdict(list)
    all_job_descriptions = {}

    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".pdf"):
                file_path = os.path.join(root, filename)
                job_title = extract_job_title(file_path)
                if job_title:
                    trait_scores, job_description = process_resume(file_path, character_traits)
                    folder_name = os.path.basename(root)
                    all_trait_scores[(folder_name, job_title)].append(trait_scores)
                    all_job_descriptions[(folder_name, job_title)] = job_description

    return all_trait_scores, all_job_descriptions

def calculate_average_scores(all_trait_scores, character_traits):
    average_scores = {}
    for (folder_name, job_title), scores in all_trait_scores.items():
        average_scores[(folder_name, job_title)] = {}
        for trait in character_traits.keys():
            trait_scores = [score.get(trait, 0) for score in scores]
            average_score = sum(trait_scores) / len(trait_scores)
            average_scores[(folder_name, job_title)][trait] = int(average_score + 0.5)

    return average_scores

# Define the character traits dictionary
character_traits = {
    "Leadership": {
        "keywords": ["lead", "manage", "supervise", "mentor", "guide", "inspire", "influence", "direct", "oversee", "coach", "motivate", "delegate"],
        "weight": 2
    },
    "Communication": {
        "keywords": ["communicate", "present", "write", "speak", "listen", "articulate", "convey", "interact", "correspond", "negotiate", "persuade", "clarify"],
        "weight": 1.5
    },
    "Teamwork": {
        "keywords": ["collaborate", "cooperate", "team player", "support", "coordinate", "contribute", "assist", "participate", "involve", "engage", "share", "partner"],
        "weight": 1.8
    },
    "Problem Solving": {
        "keywords": ["solve", "analyze", "critical thinking", "strategic", "resourceful", "troubleshoot", "innovate", "create", "develop", "design", "implement", "optimize"],
        "weight": 1.6
    },
    "Creativity": {
        "keywords": ["innovative", "creative", "original", "imaginative", "visionary", "inventive", "artistic", "expressive", "conceptual", "think outside the box", "brainstorm", "ideate"],
        "weight": 1.2
    },
    "Adaptability": {
        "keywords": ["adapt", "flexible", "versatile", "agile", "adjust", "resilient", "change", "modify", "alter", "vary", "transform", "evolve"],
        "weight": 1.4
    },
    "Work Ethic": {
        "keywords": ["dedicated", "hardworking", "committed", "reliable", "responsible", "diligent", "persistent", "tenacious", "conscientious", "meticulous", "thorough", "dependable"],
        "weight": 1.7
    },
    "Time Management": {
        "keywords": ["organize", "prioritize", "multitask", "efficient", "productive", "punctual", "schedule", "plan", "execute", "manage", "balance", "allocate"],
        "weight": 1.3
    },
    "Interpersonal Skills": {
        "keywords": ["empathy", "relationship building", "customer service", "conflict resolution", "negotiation", "diplomacy", "tact", "patience", "understanding", "compassionate", "friendly", "approachable"],
        "weight": 1.6
    },
    "Attention to Detail": {
        "keywords": ["meticulous", "thorough", "accurate", "precise", "attentive", "observant", "focused", "careful", "diligent", "scrupulous", "fastidious", "methodical"],
        "weight": 1.5
    },
    "Initiative": {
        "keywords": ["proactive", "self-starter", "motivated", "ambitious", "driven", "entrepreneurial", "eager", "energetic", "enthusiastic", "passionate", "determined", "persistent"],
        "weight": 1.4
    },
    "Analytical Thinking": {
        "keywords": ["logical", "analytical", "data-driven", "quantitative", "research", "evaluate", "assess", "examine", "investigate", "interpret", "deduce", "infer"],
        "weight": 1.3
    },
    "Emotional Intelligence": {
        "keywords": ["self-aware", "empathetic", "socially aware", "relationship management", "self-regulation", "motivation", "intuitive", "perceptive", "insightful", "reflective", "considerate", "thoughtful"],
        "weight": 1.7
    },
    "Integrity": {
        "keywords": ["honest", "ethical", "trustworthy", "transparent", "accountable", "principled", "moral", "sincere", "genuine", "reliable", "dependable", "consistent"],
        "weight": 1.9
    },
    "Resilience": {
        "keywords": ["resilient", "perseverance", "grit", "determination", "tenacity", "endurance", "stamina", "robust", "durable", "tough", "strong", "adaptable"],
        "weight": 1.6
    },
    "Cultural Awareness": {
        "keywords": ["diversity", "inclusive", "multicultural", "global", "international", "cross-cultural", "cultural sensitivity", "cultural competence", "cultural intelligence", "cultural fluency", "cultural agility", "cultural adaptability"],
        "weight": 1.4
    },
    "Years of Experience": {
        "keywords": ["years of experience", "years of expertise", "years in the field"],
        "weight": 1.5
    },
    "Programming Languages": {
        "keywords": ["Python", "Java", "C++", "JavaScript", "SQL"],
        "weight": 1.8
    },
    "Technical Skills": {
        "keywords": ["machine learning", "data analysis", "web development", "software engineering"],
        "weight": 1.6
    },
    "Office Tools": {
        "keywords": ["Microsoft Office", "Excel", "PowerPoint", "Word"],
        "weight": 1.2
    }
}

# Specify the directory containing the PDF files
directory = "/content/drive/My Drive/data"

# Process all resumes in the directory
all_trait_scores, all_job_descriptions = process_directory(directory, character_traits)

# Calculate average scores for each job title within each folder
average_scores = calculate_average_scores(all_trait_scores, character_traits)

# Create a DataFrame from the average scores
df = pd.DataFrame.from_dict(average_scores, orient='index')
df = df.fillna(0).astype(int)

# Add the job description column to the DataFrame
df['Job Description'] = df.index.map(all_job_descriptions.get)

output_directory = '/content/drive/My Drive/data'
output_file = 'character_traits_scores.csv'

df.to_csv(f'{output_directory}/{output_file}', index=True)
print(f'הקובץ {output_file} נשמר בהצלחה בתיקייה {output_directory}')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 61.1 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=9ace362791e844235ea30f50be3569acecdccfc173253a7720a6e819f76c202c
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8b622024c2d50fa4fb406d266ecdd56251f433b673542412bf20a8894e932e93
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_reso

KeyboardInterrupt: 

In [1]:
!pip install PyPDF2
!pip install nltk
!pip install pandas
!pip install sumy
!pip install spacy

import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

import spacy
nlp = spacy.load('en_core_web_sm')

import os
import PyPDF2
import re
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
import pandas as pd
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 60.0 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=e79e0b966b13d2aeb11bd9ff90a5eecff780231f66f46c4ae796f01e99e4d5b0
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6e16ab3abf8ca27ae5b4e0bae840a512dc8e5c867235193874dddad5e7475a99
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [20]:
import pandas as pd
csv_path = '/content/drive/My Drive/data/character_traits_scores.csv'
df = pd.read_csv(csv_path)
data.head()

,Unnamed: 0,Unnamed: 1,Leadership,Communication,Teamwork,Problem Solving,Creativity,Adaptability,Work Ethic,Time Management,...,Analytical Thinking,Emotional Intelligence,Integrity,Resilience,Cultural Awareness,Years of Experience,Programming Languages,Technical Skills,Office Tools,Job Description
0,DIGITAL-MEDIA,DIRECTOR,0,0,3,3,0,1,2,0,...,0,0,0,0,0,0,0,2,1,Behavioral Targeting Comscore SimilarWeb Kensh...
1,DIGITAL-MEDIA,MEDIA ACTIVITIES SPECIALIST,2,0,0,3,0,0,2,3,...,1,0,0,2,1,0,0,0,0,Summary Multi-Tasking Media Relations Results-...
2,DIGITAL-MEDIA,"SENIOR MANAGER, DIGITAL MARKETING",3,0,0,3,0,0,2,3,...,0,0,0,0,1,0,0,0,0,", State Created Customer Lifecycle Program gen..."
3,DIGITAL-MEDIA,DIGITAL PROJECT MANAGER,3,0,1,3,1,1,1,3,...,2,0,0,1,1,0,0,1,0,Created metrics to evaluate: Who is visiting 6...
4,DIGITAL-MEDIA,MONITOR TECH,2,3,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,2,Ability to perform duties under critical deadl...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1696 entries, 0 to 1695
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              1696 non-null   object
 1   Unnamed: 1              1696 non-null   object
 2   Leadership              1696 non-null   int64 
 3   Communication           1696 non-null   int64 
 4   Teamwork                1696 non-null   int64 
 5   Problem Solving         1696 non-null   int64 
 6   Creativity              1696 non-null   int64 
 7   Adaptability            1696 non-null   int64 
 8   Work Ethic              1696 non-null   int64 
 9   Time Management         1696 non-null   int64 
 10  Interpersonal Skills    1696 non-null   int64 
 11  Attention to Detail     1696 non-null   int64 
 12  Initiative              1696 non-null   int64 
 13  Analytical Thinking     1696 non-null   int64 
 14  Emotional Intelligence  1696 non-null   int64 
 15  Inte

In [21]:
# Remove the "Years of Experience" column
df = df.drop(columns=["Years of Experience"])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
!pip install gradio tensorflow pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 11.2 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl s

# here

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Extract the job title from the 'Unnamed: 1' column
df['Job Title'] = df['Unnamed: 1']

# Extract the domain from the 'Unnamed: 0' column
df['Domain'] = df['Unnamed: 0']

# Drop the 'Unnamed: 0' and 'Unnamed: 1' columns
df = df.drop(['Unnamed: 0', 'Unnamed: 1'], axis=1)

# 2. Split the data into features (X) and targets (y)
X = df.drop(['Job Title', 'Domain', 'Job Description'], axis=1)
y_title = df['Job Title']
y_domain = df['Domain']

# Perform one-hot encoding on the target variables
y_title_encoded = pd.get_dummies(y_title)
y_domain_encoded = pd.get_dummies(y_domain)

# Concatenate the encoded targets
y = pd.concat([y_title_encoded, y_domain_encoded], axis=1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 3. Build the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 4. Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# 5. Save the trained model
model.save('job_recommendation_model.h5')

# 6. Create a user interface with input validation
print("Please rate your character traits on a scale of 0 to 3:")
user_input = {}
num_questions = len(X.columns)
for i, trait in enumerate(X.columns, start=1):
    while True:
        try:
            rating = int(input(f"{trait}: "))
            if 0 <= rating <= 3:
                user_input[trait] = rating
                break
            else:
                print("Invalid input. Please enter a value between 0 and 3.")
        except ValueError:
            print("Invalid input. Please enter a valid integer.")
    print(f"{num_questions - i} questions remaining.\n")

# Convert user input to DataFrame and scale it
user_df = pd.DataFrame([user_input])
user_scaled = scaler.transform(user_df)

# 7. Use the trained model to predict the top 5 job titles and domains
predictions = model.predict(user_scaled)
num_predictions = 5

# Extract the predicted job titles and domains
predicted_titles = y_title_encoded.columns[predictions[:, :len(y_title_encoded.columns)].argsort()[-num_predictions:][::-1]]
predicted_domains = y_domain_encoded.columns[predictions[:, len(y_title_encoded.columns):].argsort()[-num_predictions:][::-1]]

# 8. Display the results to the user
print("\nTop Predicted Jobs:")
for i in range(num_predictions):
    print(f"Predicted Job {i+1}: {predicted_titles[0][i]}")
    print(f"Predicted Domain {i+1}: {predicted_domains[0][i]}\n")

Epoch 1/100
39/39 [==============================] - 1s 9ms/step - loss: 14.2234 - accuracy: 0.0000e+00 - val_loss: 13.5641 - val_accuracy: 0.0000e+00
Epoch 2/100
39/39 [==============================] - 0s 5ms/step - loss: 12.1870 - accuracy: 0.0000e+00 - val_loss: 13.0563 - val_accuracy: 0.0000e+00
Epoch 3/100
39/39 [==============================] - 0s 5ms/step - loss: 11.5730 - accuracy: 0.0000e+00 - val_loss: 13.2649 - val_accuracy: 0.0000e+00
Epoch 4/100
39/39 [==============================] - 0s 5ms/step - loss: 11.4157 - accuracy: 0.0000e+00 - val_loss: 13.6963 - val_accuracy: 0.0000e+00
Epoch 5/100
39/39 [==============================] - 0s 5ms/step - loss: 11.4054 - accuracy: 0.0000e+00 - val_loss: 14.0236 - val_accuracy: 0.0000e+00
Epoch 6/100
39/39 [==============================] - 0s 5ms/step - loss: 11.3927 - accuracy: 0.0000e+00 - val_loss: 14.4405 - val_accuracy: 0.0000e+00
Epoch 7/100
39/39 [==============================] - 0s 5ms/step - loss: 11.4515 - accuracy: 0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Test Loss: 279152.5000
Test Accuracy: 0.0000
Please rate your character traits on a scale of 0 to 3:
Leadership: 2
18 questions remaining.

Communication: 3
17 questions remaining.

Teamwork: 4
Invalid input. Please enter a value between 0 and 3.
Teamwork: 2
16 questions remaining.

Problem Solving: 1
15 questions remaining.

Creativity: 0
14 questions remaining.

Adaptability: 1
13 questions remaining.

Work Ethic: 2
12 questions remaining.

Time Management: 3
11 questions remaining.

Interpersonal Skills: 
Invalid input. Please enter a valid integer.
Interpersonal Skills: 1
10 questions remaining.

Attention to Detail: 2
9 questions remaining.

Initiative: 3
8 questions remaining.

Analytical Thinking: 3
7 questions remaining.

Emotional Intelligence: 2
6 questions remaining.

Integrity: 1
5 questions remaining.

Resilience: 0
4 questions remaining.

Cultural Awareness: 1
3 questions remaining.

Programming Languages: 2
2 questions remaining.

Technical Skills: 3
1 questions remainin

<ipython-input-22-37781d287189>:86: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predicted_titles = y_title_encoded.columns[predictions[:, :len(y_title_encoded.columns)].argsort()[-num_predictions:][::-1]]
<ipython-input-22-37781d287189>:87: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predicted_domains = y_domain_encoded.columns[predictions[:, len(y_title_encoded.columns):].argsort()[-num_predictions:][::-1]]


In [24]:
2# Filter the DataFrame to show only rows where 'Job Title' contains 'owner'
owner_jobs = df[df['Job Title'].str.contains('owner', case=False)]
owner_jobs.head(10)

,Leadership,Communication,Teamwork,Problem Solving,Creativity,Adaptability,Work Ethic,Time Management,Interpersonal Skills,Attention to Detail,...,Emotional Intelligence,Integrity,Resilience,Cultural Awareness,Programming Languages,Technical Skills,Office Tools,Job Description,Job Title,Domain
74,0,0,0,3,0,0,0,1,0,0,...,0,0,0,1,0,0,0,Qualifications Strategic marketing planning Re...,OWNER/PRESIDENT,DIGITAL-MEDIA
366,0,0,3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"I have worked in multiple restaurants, althoug...",OWNER,PUBLIC-RELATIONS
405,0,0,0,2,0,0,2,1,0,0,...,0,0,0,1,0,0,0,Responsible for successful media launch of Rad...,OWNER/CONSULTANT,PUBLIC-RELATIONS
510,2,0,0,2,1,0,1,1,2,0,...,0,1,0,1,0,0,0,Client-focused Personal Trainer for over 22 ye...,OWNER/OPERATOR,FITNESS
629,0,0,3,2,0,0,3,1,3,0,...,0,0,2,0,0,0,1,Handled customers effectively by identifying n...,OWNER,HEALTHCARE
782,3,0,2,2,0,1,0,3,2,0,...,0,0,0,0,0,0,1,"Flexible, with the ability to achieve in both ...",CHEF OWNER,CHEF
783,2,2,3,3,1,0,0,3,3,2,...,0,0,0,0,0,0,1,Summary of Skills Operations Streamlining Stra...,CO-OWNER/EXECUTIVE CHEF,CHEF
785,3,0,0,2,0,0,2,3,2,2,...,0,0,2,0,0,0,2,Built customer confidence by actively listenin...,OWNER/CHEF,CHEF
793,3,2,3,3,1,1,3,3,0,0,...,0,0,2,1,0,0,2,Scholarship Continuing education in [Topic] Sk...,CO-OWNER/MANAGER/ CHEF,CHEF
795,3,0,2,2,2,0,2,1,2,0,...,2,0,0,0,0,0,0,Areas of expertise include: Marketing & Busine...,"OWNER, PERSONAL CHEF",CHEF
